In [78]:
import pandas as pd
import numpy as np

In [79]:
bedarf_formen = pd.read_csv('data/formen_und_giesszellenbedarf_2019.csv')
prod_form_lut = pd.read_csv('data/zuweisung_produkte_und_formen.csv', 
                            dtype=dict(Produktnummer=np.uint32))
bestellungen_2019 = pd.read_csv('data/bestellungen_2019.csv').dropna()\
    .reset_index(drop=True).astype(np.uint32)
bestellungen_2020 = pd.read_csv('data/bestellungen_2020.csv').dropna()\
    .reset_index(drop=True).astype(np.uint32)
prod = prod_form_lut.pop('Produktnummer')
time_format = '%b %y'

In [80]:
#bestellungen_2019.columns

In [81]:
def prettify_orders(df):
    df.drop('Gesamt', axis=1, inplace=True)
    customer_x_prod = df.iloc[:, :2]
    orders = df.iloc[:, 2:].stack().reset_index(1)
    orders.columns = ['date', 'amt_orders']
    orders = orders.join(customer_x_prod)
    orders.date = pd.to_datetime(orders.date, 
                                 format='%b-%y').dt.strftime(time_format)
    return orders

In [82]:
orders = pd.concat([prettify_orders(bestellungen_2019), 
                    prettify_orders(bestellungen_2020)], ignore_index=True)
stacked_lut = (prod_form_lut
                .stack()
                .reset_index(1)
                .rename(columns={'level_1': 'Form', 
                                 0: 'Bedarf'}))
prod_form_lut = stacked_lut.join(prod).reset_index(drop=True).loc[:, 
                ['Produktnummer', 'Form', 'Bedarf']]
orders['attrition_by_product'] = \
    orders.Produktnummer.map(prod_form_lut[['Produktnummer', 'Bedarf']]
                       .groupby('Produktnummer').sum().Bedarf.to_dict())
orders['total_attrition'] = orders.amt_orders * orders.attrition_by_product

In [83]:
forms_cols = bedarf_formen.Form
today = pd.to_datetime('today').strftime(time_format)
prod_form_map = prod_form_lut.pivot(index='Produktnummer',
                    columns='Form', values='Bedarf')[forms_cols]
prod_giesszellenbedarf_map = prod_form_map * \
                             bedarf_formen['Gießzellenbedarf'].values.T
#prod_form_map.loc[56, 'F6']
orders_x_forms_df = orders.merge(prod_form_map.reset_index(),
                                 how='left', on='Produktnummer')
orders_x_forms_df.loc[:, forms_cols] = \
    orders_x_forms_df.loc[:, forms_cols] * orders_x_forms_df.amt_orders.values[:, np.newaxis]
form_attritions_over_time = (orders_x_forms_df
                             .groupby(['date'])[forms_cols]
                             .sum())
# sort by datetime_index
form_attritions_over_time = (form_attritions_over_time
                                 .loc[pd.to_datetime(
                                        form_attritions_over_time.index, 
                                        format=time_format)
                                 .sort_values().strftime(time_format), :])

In [90]:
duration_left = bedarf_formen['Anzahl maximaler Gießvorgänge'] - \
                bedarf_formen['Anzahl bisheriger Gießvorgänge']
next_attritions = form_attritions_over_time.loc[today:, :].cumsum()
maintenance = ((next_attritions - duration_left.values[np.newaxis, :]) < 0)
next_maintenance = maintenance.sum()
bedarf_formen['next maintenance'] = next_attritions.index[next_maintenance.clip_upper(
            len(maintenance)-1)]

/home/wilhelmk/tools/anaconda3/envs/tf-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: clip_upper(threshold) is deprecated, use clip(upper=threshold) instead
  


In [99]:
pd.to_datetime('today').strftime(time_format)

'Dec 19'